## <h1 style="color:#b4b564"> Librerias </h1> 

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",200)
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

## <h1 style="color:#b4b564"> Objetivo </h1> 

- Utilizar una estructura OLAP y OLTP para generar una TAD, enfocadas a 3 ojetivos distintos.

## <h1 style="color:#b4b564"> Secciones </h1> 
- Revisión de la información
- OPCION A : Contruir una Tabla Analitica de Datos a partir de olap y oltp para segmentar a los clientes basados en sus caracteristicas y en su comportamiento transaccional (ACTUALES)
- OPCION B: Contruir una Tabla Analitica de Datos a partir de olap y oltp para modelar la salida de un cliente y poder construir estrategias con anticipación para poder minimizar la tasa de churn [CONSIDERANDO SOLO CLIENTES CON ACTIVIDAD RECIENTE]
- OPCION C : EN CASO DE TENER UNA VARIABLE DE STATUS(EN ESE MES) DEL CLIENTE EN LA TABLA OLAP, Contruir una Tabla Analitica de Datos a partir de olap y oltp para modelar la salida de un cliente y poder construir estrategias con anticipación para poder minimizar la tasa de churn [CONSIDERANDO LA ULTIMA ACTIVIDAD REGISTRADA]

## <h1 style="color:#b4b564"> Datos </h1> 

- **fechaOper** Fecha de transacción YYYY-MM-DD .HH.Min (12 meses)
- **codCliente**	Código de cliente banco
- **rangoEdad**	Rango de edad del cliente donde: Rango1 hasta Rango6 la edad se ordena ascendentemente
- **rangoIngreso**	Rango de Ingreso del cliente donde: Rango1 hasta Rango6 la edad se ordena ascendentemente
- **flagGenero**	Flag de género del cliente donde 1: Masculino y 0: Femenino
- **flagLimaProvCliente**	Flag de procedencia del cliente donde 1: Lima/Callao y 0: Provincia
- **ubigeoCliente**	Código de ubigeo (código de procedencia) del cliente
- **codGiro**	Código de giro de transacción (rubro de consumo), el rubro puede ser: restaurantes, tiendas, hotel, etc.
- **codEstab**	Código de establecimiento o comercio, el comercio puede ser: Tienda El av. Begonias, Cineplanet av. Izaguirre, etc.
- **flagLimaProvEstab**	Flag de procedencia del establecimiento o comercio donde 1: Lima/Callao y 0: Provincia
- **ubigeoEstab**	Código de ubigeo (código de procedencia) del establecimiento o comercio
- **ctdProdAct**	Rango de cantidad de productos activos (tarjeta crédito, préstamo hipotecario, préstamo vehicular y préstamo personal) donde: Rango1 hasta Rango3 la edad se ordena ascendentemente
- **ctdProdPas**	Rango de cantidad de productos pasivos (ahorros, fondo mutuo, cts, ahorro plazo) donde: Rango1 hasta Rango3 la edad se ordena ascendentemente
- **ctdProdSeg**	Rango de cantidad de productos de seguros (oncologico, protección múltiple, vida, vehicular, etc.) donde: Rango1 hasta Rango4 la edad se ordena ascendentemente
- **flagBxi**	Flag si el cliente está afiliado a banca por internet donde 1: si lo esta y 0: no lo esta
- **saldoTcEntidad1**	Rango de saldo de tarjeta de crédito de la entidad 1 en el sistema financiero donde: Rango1 hasta Rango6 la edad se ordena ascendentemente
- **saldoTcEntidad2**	Rango de saldo de tarjeta de crédito de la entidad 2 en el sistema financiero donde: Rango1 hasta Rango5 la edad se ordena ascendentemente
- **saldoTcEntidad3**	Rango de saldo de tarjeta de crédito de la entidad 3 en el sistema financiero donde: Rango1 hasta Rango6 la edad se ordena ascendentemente
- **saldoTcEntidad4**	Rango de saldo de tarjeta de crédito de la entidad 4 en el sistema financiero donde: Rango1 hasta Rango6 la edad se ordena ascendentemente
- **ctdTrx**	Cantidad de transacción (frecuencia de consumo)
- **rankingMonto**	Ranking de prioridad de consumo(hace refenrencia a cuanto consumio)

In [ ]:
oltp=pd.read_csv("/content/drive/My Drive/G11 - Módulo I/Data/olap_oltp/oltp.csv")
olap=pd.read_csv("/content/drive/My Drive/G11 - Módulo I/Data/olap_oltp/olap.csv")

In [ ]:
oltp.shape

(1591617, 8)

In [ ]:
olap.shape

(30000, 14)

In [ ]:
oltp.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [ ]:
olap.head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,24041,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
1,9040,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN


In [ ]:
olap["rangoCtdProdAct"].value_counts()

Rango1    27952
Rango2     1866
Rango3      182
Name: rangoCtdProdAct, dtype: int64

### <h1 style="color:#b4b564"> OLAP </h1> 

In [ ]:
olap.columns=['id_codCliente', 'v_rangoEdad', 'v_rangoIngreso', 'v_flagGenero',
       'v_flagLimaProvCliente', 'v_ubigeoCliente', 'v_rangoCtdProdAct',
       'v_rangoCtdProdPas', 'v_rangoCtdProdSeg', 'v_flagBxi', 'v_saldoTcEntidad1',
       'v_saldoTcEntidad2', 'v_saldoTcEntidad3', 'v_saldoTcEntidad4']
# Asignamos un prefijo

In [ ]:
olap.duplicated().sum()


0

In [ ]:
olap.duplicated(subset=["id_codCliente"]).sum()
#duplicados por ID del cliente

0

In [ ]:
for col in olap:
    print(col)
    display(pd.DataFrame(olap[col].value_counts(1)).reset_index())
    print("\n")

id_codCliente


,index,id_codCliente
0,2047,0.000033
1,1322,0.000033
2,15629,0.000033
3,9486,0.000033
4,11535,0.000033
...,...,...
29995,17053,0.000033
29996,23198,0.000033
29997,21151,0.000033
29998,10928,0.000033




v_rangoEdad


,index,v_rangoEdad
0,Rango5,0.206826
1,Rango6,0.186454
2,Rango2,0.183315
3,Rango3,0.168821
4,Rango4,0.143873
5,Rango1,0.110710




v_rangoIngreso


...

,index,v_saldoTcEntidad2
0,Rango1,0.767116
1,Rango2,0.098117
2,Rango3,0.063375
3,Rango5,0.040256
4,Rango4,0.031136




v_saldoTcEntidad3


,index,v_saldoTcEntidad3
0,Rango1,0.878850
1,Rango6,0.030924
2,Rango2,0.024222
3,Rango3,0.023797
4,Rango5,0.021973
5,Rango4,0.020234




v_saldoTcEntidad4


,index,v_saldoTcEntidad4
0,Rango1,0.802155
1,Rango2,0.046492
2,Rango3,0.042335
3,Rango6,0.041359
4,Rango5,0.034699
5,Rango4,0.032960


In [ ]:
completitud(olap)

,columna,total,completitud
0,v_saldoTcEntidad1,6426,78.580000
1,v_saldoTcEntidad2,6426,78.580000
2,v_saldoTcEntidad3,6426,78.580000
3,v_saldoTcEntidad4,6426,78.580000
4,v_rangoIngreso,1263,95.790000
5,v_rangoEdad,57,99.810000
6,v_ubigeoCliente,4,99.986667
7,id_codCliente,0,100.000000
8,v_flagGenero,0,100.000000
9,v_flagLimaProvCliente,0,100.000000


## <h1 style="color:#b4b564"> OLTP </h1> 

In [ ]:
oltp.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [ ]:
oltp.duplicated().sum()

0

In [ ]:
oltp.columns=["d_fechaOper","id_codCliente","v_codGiro","v_codEstab","v_flagLimaProvEstab","v_ubigeoEstab","c_ctdTrx","c_ratingMonto"]

In [ ]:
completitud(oltp)

,columna,total,completitud
0,v_codGiro,92686,94.176614
1,v_ubigeoEstab,92686,94.176614
2,d_fechaOper,0,100.000000
3,id_codCliente,0,100.000000
4,v_codEstab,0,100.000000
5,v_flagLimaProvEstab,0,100.000000
6,c_ctdTrx,0,100.000000
7,c_ratingMonto,0,100.000000


In [ ]:
oltp["d_fechaOper"]

0          2017-01-29 00:00:00
1          2016-12-01 00:00:00
2          2017-06-01 00:00:00
3          2017-09-22 00:00:00
4          2017-05-05 00:00:00
                  ...         
1591612    2017-09-08 00:00:00
1591613    2017-05-21 00:00:00
1591614    2017-07-31 00:00:00
1591615    2017-05-12 00:00:00
1591616    2017-05-20 00:00:00
Name: d_fechaOper, Length: 1591617, dtype: object

In [ ]:
oltp["d_fechaOper"]=pd.to_datetime(oltp["d_fechaOper"])

In [ ]:
oltp["d_fechaOper"].min()

Timestamp('2016-11-01 00:00:00')

In [ ]:
oltp["d_fechaOper"].max()

Timestamp('2017-10-31 00:00:00')

In [ ]:
oltp

,d_fechaOper,id_codCliente,v_codGiro,v_codEstab,v_flagLimaProvEstab,v_ubigeoEstab,c_ctdTrx,c_ratingMonto
0,2017-01-29,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05,16635,75.0,31043,0,81.0,1,0.047386
...,...,...,...,...,...,...,...,...
1591612,2017-09-08,7404,114.0,30314,1,157.0,1,0.000901
1591613,2017-05-21,12253,110.0,29035,1,165.0,1,0.000706
1591614,2017-07-31,16096,75.0,4323,1,171.0,1,0.005984
1591615,2017-05-12,8852,110.0,51574,1,175.0,1,0.000613


In [ ]:
oltp[["id_codCliente","d_fechaOper"]].sort_values(by=["id_codCliente","d_fechaOper"])

,id_codCliente,d_fechaOper
1331328,1,2016-11-20
816568,1,2016-12-22
1461495,1,2016-12-28
399638,1,2016-12-31
580944,1,2017-01-05
...,...,...
1590630,30000,2017-09-24
1231164,30000,2017-09-30
853976,30000,2017-10-05
1153259,30000,2017-10-15


In [ ]:
oltp[["id_codCliente","c_ratingMonto"]].groupby(["id_codCliente"]).count()

,c_ratingMonto
id_codCliente,
1,36
2,119
3,13
4,181
5,41
...,...
29996,22
29997,32
29998,24


In [ ]:
#Número de transacciones por cliente
oltp[["id_codCliente","c_ratingMonto"]].groupby(["id_codCliente"]).count()["c_ratingMonto"].mean()

53.0539

In [ ]:
oltp[["id_codCliente","c_ratingMonto"]].groupby(["id_codCliente"]).count().describe(percentiles=np.arange(0.1,1.1,.1))
# Hay registros con 1 transacción y otras con hastan 730 transacciones 

,c_ratingMonto
count,30000.000000
mean,53.053900
std,61.829447
min,1.000000
10%,9.000000
20%,13.000000
30%,18.000000
40%,24.000000
50%,31.000000
60%,41.000000


In [ ]:
#Meses con más transacciones
oltp["d_fechaOper"].map(lambda x:x.month_name()).value_counts(1)

July         0.096457
December     0.093796
August       0.089826
September    0.085834
March        0.084836
October      0.084801
May          0.084296
June         0.083979
January      0.078718
February     0.077480
April        0.071175
November     0.068800
Name: d_fechaOper, dtype: float64

In [ ]:
#Dia de la semana con más transacciones
oltp["d_fechaOper"].map(lambda x:x.day_name()).value_counts(1)

Saturday     0.174803
Friday       0.150828
Sunday       0.143731
Thursday     0.136233
Wednesday    0.135506
Tuesday      0.133307
Monday       0.125592
Name: d_fechaOper, dtype: float64

In [ ]:
oltp.describe(percentiles=np.arange(0.1,1.1,.1))

,id_codCliente,v_codGiro,v_codEstab,v_flagLimaProvEstab,v_ubigeoEstab,c_ctdTrx,c_ratingMonto
count,1.591617e+06,1.498931e+06,1.591617e+06,1.591617e+06,1.498931e+06,1.591617e+06,1.591617e+06
mean,1.334583e+04,1.048248e+02,2.453260e+04,8.084175e-01,1.747467e+02,1.072021e+00,1.318383e-02
std,8.493548e+03,3.920585e+01,1.990371e+04,3.935464e-01,6.856421e+01,3.465646e-01,3.236438e-02
min,1.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,2.984905e-06
10%,2.236000e+03,7.200000e+01,8.320000e+02,0.000000e+00,1.360000e+02,1.000000e+00,6.793478e-04
20%,4.733000e+03,7.500000e+01,3.411000e+03,1.000000e+00,1.470000e+02,1.000000e+00,1.261830e-03
30%,7.258000e+03,7.500000e+01,7.427000e+03,1.000000e+00,1.500000e+02,1.000000e+00,1.981155e-03
40%,9.774000e+03,8.500000e+01,1.884500e+04,1.000000e+00,1.570000e+02,1.000000e+00,2.908789e-03
50%,1.258100e+04,1.080000e+02,2.266500e+04,1.000000e+00,1.650000e+02,1.000000e+00,4.199563e-03
60%,1.551700e+04,1.100000e+02,2.931100e+04,1.000000e+00,1.660000e+02,1.000000e+00,6.089940e-03


In [ ]:
completitud(oltp)

,columna,total,completitud
0,v_codGiro,92686,94.176614
1,v_ubigeoEstab,92686,94.176614
2,d_fechaOper,0,100.000000
3,id_codCliente,0,100.000000
4,v_codEstab,0,100.000000
5,v_flagLimaProvEstab,0,100.000000
6,c_ctdTrx,0,100.000000
7,c_ratingMonto,0,100.000000


In [ ]:
#oltp=oltp.dropna()

In [ ]:
oltp["d_fechaOper"].max()

Timestamp('2017-10-31 00:00:00')

In [ ]:
id_cliente=list(oltp["id_codCliente"].unique())

In [ ]:
len(id_cliente)

30000

## <h1 style="color:#b4b564"> OPCION A </h1>

**OBJETIVO**
- Contruir una Tabla Analitica de Datos a partir de olap y oltp para segmentar a los clientes basados en sus caracteristicas y en su comportamiento transaccional (ACTUALES)

In [ ]:
oltp["d_fechaOper"].min()

Timestamp('2016-11-01 00:00:00')

In [ ]:
oltp["d_fechaOper"].max()

Timestamp('2017-10-31 00:00:00')

In [ ]:
import datetime
#Incorporar información transaccional de 7 meses atrás 
start_date = datetime.datetime(2017,3,1)
end_date = datetime.datetime(2017, 10, 1)

In [ ]:
#Solo tomamos aquellos clientes que tuvieron alguna transaccion en estos ultimos 7 meses
data=oltp[oltp["d_fechaOper"].map(lambda x:end_date>=x>=start_date)]

In [ ]:
data["key"]=data["d_fechaOper"].map(lambda x:"-".join([str(x.year),str(x.month)]))


In [ ]:
data["c_ctdTrx"].describe()

count    954881.000000
mean          1.071476
std           0.344435
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          21.000000
Name: c_ctdTrx, dtype: float64

In [ ]:
data["c_ratingMonto"].describe()

count    954881.000000
mean          0.013248
std           0.032430
min           0.000003
25%           0.001606
50%           0.004221
75%           0.011593
max           1.000000
Name: c_ratingMonto, dtype: float64

In [ ]:
#Queremos agregar información mensual, por lo cual agrupamos las transacciones diarias a mensuales, tomando la media
#Solo se tomaran las variables de monto de consumo-rating y la frecuencia de consumo
data=data[["key","id_codCliente","c_ctdTrx","c_ratingMonto"]].groupby(["key","id_codCliente"]).mean().reset_index()
data

,key,id_codCliente,c_ctdTrx,c_ratingMonto
0,2017-10,5,1.000000,0.004711
1,2017-10,9,1.000000,0.011394
2,2017-10,13,1.500000,0.000328
3,2017-10,19,1.000000,0.003477
4,2017-10,29,1.000000,0.003943
...,...,...,...,...
170141,2017-9,29996,1.000000,0.018570
170142,2017-9,29997,1.142857,0.029595
170143,2017-9,29998,1.000000,0.013855
170144,2017-9,29999,1.000000,0.092251


In [ ]:
#Los meses los convertimos en variables
data=data.pivot_table(["c_ctdTrx",'c_ratingMonto'], ["id_codCliente"], 'key')

In [ ]:
data

c_ctdTrx                                                  \
key            2017-10 2017-3 2017-4 2017-5    2017-6    2017-7 2017-8   
id_codCliente                                                            
1                  NaN    1.0    1.0    1.0  1.000000  1.000000    1.0   
2                  NaN    1.0    1.0    1.0  1.090909  1.000000    1.0   
3                  NaN    1.0    NaN    1.0  1.000000       NaN    NaN   
4                  NaN    1.0    1.0    1.0  1.166667  1.095238    1.0   
5                  1.0    1.0    1.0    1.0  1.000000  1.000000    NaN   
...                ...    ...    ...    ...       ...       ...    ...   
29996              NaN    NaN    NaN    NaN  1.000000  1.000000    1.0   
29997              NaN    NaN    NaN    1.0  1.100000  1.000000    1.0   
29998              NaN    NaN    NaN    1.0  1.000000  1.000000    1.0   
29999              NaN    NaN    NaN    1.0  1.000000  1.000000    NaN   
30000              NaN    NaN    NaN    1.0  1.000000  1.000000    1.0   

                        c_ratingMonto                                          \
key              2017-9       2017-10    2017-3    2017-4    2017-5    2017-6   
id_codCliente                                                                   
1                   NaN           NaN  0.030324  0.006212  0.038929  0.015218   
2              1.066667           NaN  0.003015  0.004256  0.002868  0.003436   
3              1.000000           NaN  0.011377       NaN  0.042051  0.107143   
4              1.000000           NaN  0.002168  0.002151  0.002507  0.002794   
5              1.000000      0.004711  0.000919  0.004432  0.022142  0.014337   
...                 ...           ...       ...       ...       ...       ...   
29996          1.000000           NaN       NaN       NaN       NaN  0.052892   
29997          1.142857           NaN       NaN       NaN  0.009680  0.023787   
29998          1.000000           NaN       NaN       NaN  0.019560  0.022005   
29999          1.000000           NaN       NaN       NaN  0.092251  0.011070   
30000          1.000000           NaN       NaN       NaN  0.018382  0.012255   

                                             
key              2017-7    2017-8    2017-9  
id_codCliente                                
1              0.007129  0.036963       NaN  
2              0.002814  0.002498  0.008733  
3                   NaN       NaN  0.041763  
4              0.006016  0.003483  0.004876  
5              0.085495       NaN  0.012017  
...                 ...       ...       ...  
29996          0.012496  0.026918  0.018570  
29997          0.018914  0.015176  0.029595  
29998          0.040342  0.010758  0.013855  
29999          0.092251       NaN  0.092251  
30000          0.031950  0.018382  0.054075  

[29967 rows x 16 columns]

In [ ]:
#renombramos las columnas
data.columns=["_".join(x) for x in data.columns]

In [ ]:
data.reset_index(inplace=True)


In [ ]:
#Número de meses sin movimientos
data.isnull().sum(1).value_counts(1)

2     0.314713
4     0.188240
6     0.156606
8     0.118464
0     0.096339
10    0.076551
12    0.039010
14    0.010078
dtype: float64

In [ ]:
data["n_missings"]= data.isnull().sum(1)

In [ ]:
data= data[data["n_missings"]<=8]

In [ ]:
#Los clientes que no tuvieron transacciones en un determinado mes se indica que la frecuencia y monto fue cero
data=data.fillna(0)

In [ ]:
olap.shape

(30000, 14)

In [ ]:
data.shape

(26202, 18)

In [ ]:
#agregamos las caracteristicas de los clientes
df_all=data.merge(olap,on="id_codCliente",how="right")

In [ ]:
df_all

,id_codCliente,c_ctdTrx_2017-10,c_ctdTrx_2017-3,c_ctdTrx_2017-4,c_ctdTrx_2017-5,c_ctdTrx_2017-6,c_ctdTrx_2017-7,c_ctdTrx_2017-8,c_ctdTrx_2017-9,c_ratingMonto_2017-10,c_ratingMonto_2017-3,c_ratingMonto_2017-4,c_ratingMonto_2017-5,c_ratingMonto_2017-6,c_ratingMonto_2017-7,c_ratingMonto_2017-8,c_ratingMonto_2017-9,n_missings,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4
0,24041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
1,9040,0.000000,1.0,1.000000,1.000000,1.142857,1.0,1.25,1.100,0.000000,0.009238,0.012988,0.011378,0.014927,0.011795,0.006368,0.005615,2.0,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,0.000000,1.0,1.000000,1.000000,1.000000,1.0,1.00,1.000,0.000000,0.004680,0.009059,0.014217,0.003020,0.015601,0.021490,0.004781,2.0,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,1.666667,1.0,1.133333,1.184211,1.148148,1.0,1.00,1.075,0.001319,0.002042,0.001630,0.002570,0.003230,0.001483,0.001285,0.002272,0.0,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,0.000000,1.0,1.000000,1.000000,0.000000,0.0,1.00,1.000,0.000000,0.024334,0.024105,0.010331,0.000000,0.000000,0.032369,0.016529,6.0,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,253,0.000000,2.0,1.000000,1.000000,1.000000,1.0,1.00,1.000,0.000000,0.035156,0.036186,0.001989,0.023449,0.001367,0.016264,0.023807,2.0,Rango6,Rango2,0,1,271.0,Rango1,Rango3,Rango1,0,Rango1,Rango2,Rango1,Rango1
29996,13378,0.000000,1.0,0.000000,0.000000,1.000000,1.0,1.00,1.000,0.000000,0.055504,0.000000,0.000000,0.162812,0.013876,0.037928,0.113321,6.0,Rango6,Rango1,0,1,271.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango4
29997,12503,0.000000,1.0,0.000000,0.000000,0.000000,1.0,1.00,1.000,0.000000,0.012571,0.000000,0.000000,0.000000,0.004461,0.004055,0.015004,8.0,Rango3,Rango1,1,1,255.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
29998,22468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango5,Rango3,1,1,255.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1


In [ ]:
#Total de clientes que tuvieron al menos una transaccion en los ultimos 7 meses
len(data["id_codCliente"].unique())

26202

In [ ]:
#total clientes registrados
len(df_all["id_codCliente"].unique())

30000

In [ ]:
#total clientes registrados
len(olap["id_codCliente"].unique())

30000

In [ ]:
df_all.shape

(30000, 31)

In [ ]:
#Clientes que no serán aptos para realizar la segmentacion, de los clientes actuales
#posibles clientes que dejaron de realizar movimientos meses atras
df_all[df_all["c_ctdTrx_2017-3"].isnull()]

,id_codCliente,c_ctdTrx_2017-10,c_ctdTrx_2017-3,c_ctdTrx_2017-4,c_ctdTrx_2017-5,c_ctdTrx_2017-6,c_ctdTrx_2017-7,c_ctdTrx_2017-8,c_ctdTrx_2017-9,c_ratingMonto_2017-10,c_ratingMonto_2017-3,c_ratingMonto_2017-4,c_ratingMonto_2017-5,c_ratingMonto_2017-6,c_ratingMonto_2017-7,c_ratingMonto_2017-8,c_ratingMonto_2017-9,n_missings,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4
0,24041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
11,7124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango5,Rango2,0,1,273.0,Rango1,Rango2,Rango1,0,NaN,NaN,NaN,NaN
16,7479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango5,Rango2,1,1,268.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
31,13327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango4,Rango4,1,1,77.0,Rango1,Rango1,Rango1,1,Rango4,Rango1,Rango1,Rango1
33,9728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango3,Rango3,0,0,357.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29964,18669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango5,Rango1,1,0,399.0,Rango2,Rango2,Rango2,1,Rango5,Rango1,Rango1,Rango1
29967,16402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango3,Rango4,1,0,116.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN
29974,25728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango6,Rango1,0,1,246.0,Rango1,Rango1,Rango2,1,Rango1,Rango2,Rango1,Rango1
29984,246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rango6,Rango3,0,1,246.0,Rango1,Rango1,Rango1,0,Rango1,Rango1,Rango1,Rango1


In [ ]:
tabla_final=df_all[~df_all["c_ctdTrx_2017-3"].isnull()]

In [ ]:
tabla_final

,id_codCliente,c_ctdTrx_2017-10,c_ctdTrx_2017-3,c_ctdTrx_2017-4,c_ctdTrx_2017-5,c_ctdTrx_2017-6,c_ctdTrx_2017-7,c_ctdTrx_2017-8,c_ctdTrx_2017-9,c_ratingMonto_2017-10,c_ratingMonto_2017-3,c_ratingMonto_2017-4,c_ratingMonto_2017-5,c_ratingMonto_2017-6,c_ratingMonto_2017-7,c_ratingMonto_2017-8,c_ratingMonto_2017-9,n_missings,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4
1,9040,0.000000,1.0,1.000000,1.000000,1.142857,1.000000,1.250000,1.100,0.000000,0.009238,0.012988,0.011378,0.014927,0.011795,0.006368,0.005615,2.0,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,0.000000,0.004680,0.009059,0.014217,0.003020,0.015601,0.021490,0.004781,2.0,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,1.666667,1.0,1.133333,1.184211,1.148148,1.000000,1.000000,1.075,0.001319,0.002042,0.001630,0.002570,0.003230,0.001483,0.001285,0.002272,0.0,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,0.000000,1.0,1.000000,1.000000,0.000000,0.000000,1.000000,1.000,0.000000,0.024334,0.024105,0.010331,0.000000,0.000000,0.032369,0.016529,6.0,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN
5,9649,1.000000,1.0,1.333333,1.190476,1.000000,1.222222,1.333333,1.000,0.000658,0.004218,0.035157,0.006545,0.004474,0.009739,0.003184,0.003426,0.0,Rango5,Rango6,1,1,273.0,Rango2,Rango2,Rango2,1,Rango2,Rango1,Rango1,Rango1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,6494,0.000000,1.0,1.000000,1.333333,1.000000,1.000000,1.000000,1.500,0.000000,0.030238,0.009509,0.007322,0.011777,0.011326,0.006792,0.010694,2.0,Rango5,Rango2,0,1,271.0,Rango1,Rango1,Rango1,1,Rango4,Rango1,Rango1,Rango1
29995,253,0.000000,2.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,0.000000,0.035156,0.036186,0.001989,0.023449,0.001367,0.016264,0.023807,2.0,Rango6,Rango2,0,1,271.0,Rango1,Rango3,Rango1,0,Rango1,Rango2,Rango1,Rango1
29996,13378,0.000000,1.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000,0.000000,0.055504,0.000000,0.000000,0.162812,0.013876,0.037928,0.113321,6.0,Rango6,Rango1,0,1,271.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango4
29997,12503,0.000000,1.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.000,0.000000,0.012571,0.000000,0.000000,0.000000,0.004461,0.004055,0.015004,8.0,Rango3,Rango1,1,1,255.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1


In [ ]:
#Tabla lista para limpieza en caracteristicas y evaluación de missings
completitud(tabla_final)

,columna,total,completitud
0,v_saldoTcEntidad4,5477,79.097015
1,v_saldoTcEntidad2,5477,79.097015
2,v_saldoTcEntidad1,5477,79.097015
3,v_saldoTcEntidad3,5477,79.097015
4,v_rangoIngreso,1131,95.683536
5,v_rangoEdad,46,99.824441
6,v_ubigeoCliente,4,99.984734
7,v_flagBxi,0,100.000000
8,v_rangoCtdProdSeg,0,100.000000
9,v_rangoCtdProdPas,0,100.000000


## <h1 style="color:#b4b564"> OPCION B </h1>

**OBJETIVO**
- Contruir una Tabla Analitica de Datos a partir de olap y oltp para modelar la salida de un cliente y poder construir estrategias con anticipación para poder minimizar la tasa de churn [CONSIDERANDO SOLO CLIENTES CON ACTIVIDAD RECIENTE]

In [ ]:
df_all=tabla_final.copy()
#Analizamos un mes atrás. Si hace un mes no ha hecho transacciones, ya no lo consideramos como cliente. 

In [ ]:
df_all.describe(percentiles=np.arange(0.1,1.1,.1))

,id_codCliente,c_ctdTrx_2017-10,c_ctdTrx_2017-3,c_ctdTrx_2017-4,c_ctdTrx_2017-5,c_ctdTrx_2017-6,c_ctdTrx_2017-7,c_ctdTrx_2017-8,c_ctdTrx_2017-9,c_ratingMonto_2017-10,c_ratingMonto_2017-3,c_ratingMonto_2017-4,c_ratingMonto_2017-5,c_ratingMonto_2017-6,c_ratingMonto_2017-7,c_ratingMonto_2017-8,c_ratingMonto_2017-9,n_missings,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_flagBxi
count,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26202.000000,26198.000000,26202.000000
mean,14896.168766,0.174636,0.878136,0.861848,0.917065,0.916642,0.949597,0.933890,0.910663,0.002535,0.019223,0.018191,0.020638,0.019788,0.023518,0.021559,0.019886,3.739562,0.548699,0.793107,237.608176,0.828677
std,8687.136667,0.427504,0.454523,0.464919,0.424794,0.423119,0.397274,0.415468,0.440256,0.014823,0.036924,0.036432,0.037641,0.035380,0.040748,0.039497,0.038110,2.453419,0.497632,0.405086,76.886277,0.376798
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
10%,2907.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.000000,0.000000
20%,5901.200000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.001544,0.001004,0.002414,0.002370,0.003213,0.002666,0.002125,2.000000,0.000000,0.000000,212.000000,1.000000
30%,8842.300000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.003592,0.003175,0.004261,0.004219,0.005318,0.004589,0.003994,2.000000,0.000000,1.000000,246.000000,1.000000
40%,11798.400000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.005615,0.005055,0.006373,0.006326,0.007826,0.006753,0.006001,2.000000,0.000000,1.000000,253.000000,1.000000
50%,14841.500000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.008129,0.007421,0.009072,0.008901,0.010979,0.009594,0.008613,4.000000,1.000000,1.000000,258.000000,1.000000
60%,17839.600000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.011430,0.010733,0.012725,0.012434,0.015458,0.013389,0.012176,4.000000,1.000000,1.000000,270.000000,1.000000


In [ ]:
#Restringimos los registros a solo aquellos clientes que en los ultimos seis meses han tenido movimientos en al menos tres meses
#Y evitar a aquellos que son nuevos o los que han dejado de tener movimiento tiempo atras
df_all=df_all[df_all[['c_ctdTrx_2017-10', 'c_ctdTrx_2017-3',
       'c_ctdTrx_2017-4', 'c_ctdTrx_2017-5', 'c_ctdTrx_2017-6',
       'c_ctdTrx_2017-7', 'c_ctdTrx_2017-8', 'c_ctdTrx_2017-9']].apply(lambda x:sum([y!=0 for y in x])>3,axis=1)]

In [ ]:
df_all.shape

(26202, 31)

In [ ]:
df_all[(df_all["c_ctdTrx_2017-10"]==0)&(df_all["c_ctdTrx_2017-9"]==0)]
#3660 no hicieron transacciones

,id_codCliente,c_ctdTrx_2017-10,c_ctdTrx_2017-3,c_ctdTrx_2017-4,c_ctdTrx_2017-5,c_ctdTrx_2017-6,c_ctdTrx_2017-7,c_ctdTrx_2017-8,c_ctdTrx_2017-9,c_ratingMonto_2017-10,c_ratingMonto_2017-3,c_ratingMonto_2017-4,c_ratingMonto_2017-5,c_ratingMonto_2017-6,c_ratingMonto_2017-7,c_ratingMonto_2017-8,c_ratingMonto_2017-9,n_missings,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4
7,2486,0.0,0.000000,2.000000,1.0,0.0,1.333333,1.0,0.0,0.0,0.000000,0.109489,0.033629,0.000000,0.147636,0.042101,0.0,8.0,Rango5,Rango1,0,1,241.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
14,29390,0.0,1.000000,1.000000,0.0,0.0,1.000000,1.0,0.0,0.0,0.038552,0.022817,0.000000,0.000000,0.052242,0.022817,0.0,8.0,Rango3,Rango2,1,1,273.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
22,17221,0.0,0.000000,1.000000,1.0,1.0,1.333333,0.0,0.0,0.0,0.000000,0.051402,0.032043,0.133511,0.105919,0.000000,0.0,8.0,Rango3,Rango1,1,1,273.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
30,20314,0.0,1.000000,0.000000,1.0,1.0,1.200000,0.0,0.0,0.0,0.032530,0.000000,0.073194,0.011573,0.024711,0.000000,0.0,8.0,Rango5,Rango4,0,0,88.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
34,25195,0.0,1.000000,1.000000,1.0,0.0,1.000000,0.0,0.0,0.0,0.084092,0.088101,0.009103,0.000000,0.090377,0.000000,0.0,8.0,Rango2,Rango1,0,1,272.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29954,3064,0.0,1.000000,1.000000,1.0,1.0,1.000000,1.0,0.0,0.0,0.014786,0.020109,0.011374,0.009099,0.009099,0.009164,0.0,4.0,Rango6,Rango2,1,1,81.0,Rango1,Rango1,Rango1,0,Rango1,Rango1,Rango1,Rango1
29955,22084,0.0,1.000000,0.000000,1.0,0.0,1.000000,1.0,0.0,0.0,0.009010,0.000000,0.003604,0.000000,0.023214,0.006693,0.0,8.0,Rango3,Rango2,0,1,246.0,Rango1,Rango1,Rango2,1,NaN,NaN,NaN,NaN
29963,10674,0.0,1.333333,1.000000,1.0,0.0,1.000000,1.0,0.0,0.0,0.021842,0.022080,0.020655,0.000000,0.081197,0.026353,0.0,6.0,Rango4,Rango1,0,1,246.0,Rango1,Rango1,Rango1,0,Rango1,Rango1,Rango1,Rango1
29979,13936,0.0,1.000000,1.000000,1.0,0.0,1.000000,1.0,0.0,0.0,0.064286,0.092857,0.028571,0.000000,0.021429,0.050000,0.0,6.0,Rango2,Rango1,1,1,81.0,Rango1,Rango2,Rango1,1,NaN,NaN,NaN,NaN


In [ ]:
#Se considerará como estatus inactivo a las personas que no transaccionaron este mes 
#y el anterior (2017-10,2017-9)
df_all[(df_all["c_ctdTrx_2017-10"]==0)&(df_all["c_ctdTrx_2017-9"]==0)].shape[0]/df_all.shape[0]

0.1396839935882757

In [ ]:
#construimos nuestra target (0-inactivos, 1-activos)
#Tasa de evento
df_all[['c_ctdTrx_2017-10', 'c_ctdTrx_2017-9']].apply(lambda x:sum([y==0 for y in x])!=2,axis=1).value_counts(1)

True     0.860316
False    0.139684
dtype: float64

In [ ]:
df_all["target"]=df_all[['c_ctdTrx_2017-10', 'c_ctdTrx_2017-9']].apply(lambda x:sum([y==0 for y in x])==2,axis=1)*1
# multiplicmos por 1 para que nos regrese 1 ó 0, no TRue y false

In [ ]:
df_all["target"].value_counts()

0    22542
1     3660
Name: target, dtype: int64

In [ ]:
#Eliminamos aquellas variables con las cuales se construyo la target
df_all=df_all.drop(columns=['c_ctdTrx_2017-10', 'c_ctdTrx_2017-9',"c_ratingMonto_2017-10","c_ctdTrx_2017-9"])

In [ ]:
df_all

,id_codCliente,c_ctdTrx_2017-3,c_ctdTrx_2017-4,c_ctdTrx_2017-5,c_ctdTrx_2017-6,c_ctdTrx_2017-7,c_ctdTrx_2017-8,c_ratingMonto_2017-3,c_ratingMonto_2017-4,c_ratingMonto_2017-5,c_ratingMonto_2017-6,c_ratingMonto_2017-7,c_ratingMonto_2017-8,c_ratingMonto_2017-9,n_missings,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4,target
1,9040,1.0,1.000000,1.000000,1.142857,1.000000,1.250000,0.009238,0.012988,0.011378,0.014927,0.011795,0.006368,0.005615,2.0,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1,0
2,18233,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.004680,0.009059,0.014217,0.003020,0.015601,0.021490,0.004781,2.0,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1,0
3,7226,1.0,1.133333,1.184211,1.148148,1.000000,1.000000,0.002042,0.001630,0.002570,0.003230,0.001483,0.001285,0.002272,0.0,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2,0
4,21617,1.0,1.000000,1.000000,0.000000,0.000000,1.000000,0.024334,0.024105,0.010331,0.000000,0.000000,0.032369,0.016529,6.0,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN,0
5,9649,1.0,1.333333,1.190476,1.000000,1.222222,1.333333,0.004218,0.035157,0.006545,0.004474,0.009739,0.003184,0.003426,0.0,Rango5,Rango6,1,1,273.0,Rango2,Rango2,Rango2,1,Rango2,Rango1,Rango1,Rango1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,6494,1.0,1.000000,1.333333,1.000000,1.000000,1.000000,0.030238,0.009509,0.007322,0.011777,0.011326,0.006792,0.010694,2.0,Rango5,Rango2,0,1,271.0,Rango1,Rango1,Rango1,1,Rango4,Rango1,Rango1,Rango1,0
29995,253,2.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.035156,0.036186,0.001989,0.023449,0.001367,0.016264,0.023807,2.0,Rango6,Rango2,0,1,271.0,Rango1,Rango3,Rango1,0,Rango1,Rango2,Rango1,Rango1,0
29996,13378,1.0,0.000000,0.000000,1.000000,1.000000,1.000000,0.055504,0.000000,0.000000,0.162812,0.013876,0.037928,0.113321,6.0,Rango6,Rango1,0,1,271.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango4,0
29997,12503,1.0,0.000000,0.000000,0.000000,1.000000,1.000000,0.012571,0.000000,0.000000,0.000000,0.004461,0.004055,0.015004,8.0,Rango3,Rango1,1,1,255.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,0


In [ ]:
df_all.fillna("Missings")
# Para cambiar la leyenda NaN por Missings

## <h1 style="color:#b4b564"> OPCION C </h1>

**OBJETIVO**
- EN CASO DE TENER UNA VARIABLE DE STATUS(EN ESE MES) DEL CLIENTE EN LA TABLA OLAP, Contruir una Tabla Analitica de Datos a partir de olap y oltp para modelar la salida de un cliente y poder construir estrategias con anticipación para poder minimizar la tasa de churn [CONSIDERANDO LA ULTIMA ACTIVIDAD REGISTRADA]

In [ ]:
olap

,id_codCliente,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4
0,24041,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
1,9040,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,253,Rango6,Rango2,0,1,271.0,Rango1,Rango3,Rango1,0,Rango1,Rango2,Rango1,Rango1
29996,13378,Rango6,Rango1,0,1,271.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango4
29997,12503,Rango3,Rango1,1,1,255.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
29998,22468,Rango5,Rango3,1,1,255.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1


In [ ]:
import random

In [ ]:
olap["target"]=pd.Series(random.choices([1,0], weights=[.7, .3], k=len(olap)))

In [ ]:
olap["target"].value_counts(1)

1    0.7024
0    0.2976
Name: target, dtype: float64

In [ ]:
data=oltp.copy()

In [ ]:
data["key"]=data["d_fechaOper"].map(lambda x:"-".join([str(x.year),str(x.month)]))

In [ ]:
#Queremos agregar información mensual, por lo cual agrupamos las transacciones diarias a mensuales, tomando la media
#Solo se tomaran las variables de monto de consumo-rating y la frecuencia de consumo
data=data[["key","id_codCliente","c_ctdTrx","c_ratingMonto"]].groupby(["key","id_codCliente"]).mean().reset_index()

In [ ]:
#Información mas reciente
data=data.sort_values(["id_codCliente","key"],ascending=False)

In [ ]:
data

,key,id_codCliente,c_ctdTrx,c_ratingMonto
275583,2017-9,30000,1.0,0.054075
251704,2017-8,30000,1.0,0.018382
227168,2017-7,30000,1.0,0.031950
202458,2017-6,30000,1.0,0.012255
178744,2017-5,30000,1.0,0.018382
...,...,...,...,...
109736,2017-3,1,1.0,0.030324
87626,2017-2,1,1.0,0.009807
42535,2017-1,1,1.0,0.022884
20020,2016-12,1,1.0,0.017712


In [ ]:
def add_month(df):
    df=df.reset_index(drop=True)
    aux=df.reindex(range(0,12), fill_value=np.nan)
    aux["id_codCliente"]=aux["id_codCliente"].fillna(method="ffill")
    aux=aux.fillna(0)
    return aux

In [ ]:
data=data.groupby('id_codCliente', group_keys=False).apply(add_month)

In [ ]:
data.head(25)

,key,id_codCliente,c_ctdTrx,c_ratingMonto
0,2017-8,1.0,1.000000,0.036963
1,2017-7,1.0,1.000000,0.007129
2,2017-6,1.0,1.000000,0.015218
3,2017-5,1.0,1.000000,0.038929
4,2017-4,1.0,1.000000,0.006212
5,2017-3,1.0,1.000000,0.030324
6,2017-2,1.0,1.000000,0.009807
7,2017-1,1.0,1.000000,0.022884
8,2016-12,1.0,1.000000,0.017712
9,2016-11,1.0,1.000000,0.012931


In [ ]:
data["id_codCliente"].value_counts()

1.0        12
19997.0    12
20009.0    12
20008.0    12
20007.0    12
           ..
9996.0     12
9995.0     12
9994.0     12
9993.0     12
30000.0    12
Name: id_codCliente, Length: 30000, dtype: int64

In [ ]:
data["key"]=[f"month_{i}" for i in range(1,13)]*int(data.shape[0]/12)

In [ ]:
data
#Debemos eliminar los primeros dos registros

,key,id_codCliente,c_ctdTrx,c_ratingMonto
0,month_1,1.0,1.0,0.036963
1,month_2,1.0,1.0,0.007129
2,month_3,1.0,1.0,0.015218
3,month_4,1.0,1.0,0.038929
4,month_5,1.0,1.0,0.006212
...,...,...,...,...
7,month_8,30000.0,0.0,0.000000
8,month_9,30000.0,0.0,0.000000
9,month_10,30000.0,0.0,0.000000
10,month_11,30000.0,0.0,0.000000


In [ ]:
data=data.pivot_table(["c_ctdTrx",'c_ratingMonto'], ["id_codCliente"], 'key')

data.columns=["_".join(x) for x in data.columns]

data.reset_index(inplace=True)

In [ ]:
data

,id_codCliente,c_ctdTrx_month_1,c_ctdTrx_month_10,c_ctdTrx_month_11,c_ctdTrx_month_12,c_ctdTrx_month_2,c_ctdTrx_month_3,c_ctdTrx_month_4,c_ctdTrx_month_5,c_ctdTrx_month_6,c_ctdTrx_month_7,c_ctdTrx_month_8,c_ctdTrx_month_9,c_ratingMonto_month_1,c_ratingMonto_month_10,c_ratingMonto_month_11,c_ratingMonto_month_12,c_ratingMonto_month_2,c_ratingMonto_month_3,c_ratingMonto_month_4,c_ratingMonto_month_5,c_ratingMonto_month_6,c_ratingMonto_month_7,c_ratingMonto_month_8,c_ratingMonto_month_9
0,1.0,1.000000,1.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,0.036963,0.012931,0.000000,0.000000,0.007129,0.015218,0.038929,0.006212,0.030324,0.009807,0.022884,0.017712
1,2.0,1.066667,1.125,1.0,1.166667,1.0,1.000000,1.090909,1.000000,1.000000,1.0,1.000000,1.0,0.008733,0.001440,0.002662,0.002839,0.002498,0.002814,0.003436,0.002868,0.004256,0.003015,0.004821,0.011069
2,3.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.0,0.041763,0.000000,0.000000,0.000000,0.107143,0.042051,0.011377,0.008353,0.018433,0.052707,0.070853,0.000000
3,4.0,1.000000,1.000,1.0,1.066667,1.0,1.095238,1.166667,1.000000,1.000000,1.0,1.300000,1.0,0.004876,0.002249,0.004774,0.003082,0.003483,0.006016,0.002794,0.002507,0.002151,0.002168,0.017737,0.003091
4,5.0,1.000000,1.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.090909,1.0,0.012017,0.016805,0.000000,0.000000,0.085495,0.014337,0.022142,0.004432,0.000919,0.001937,0.011503,0.008077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.285714,0.000000,0.0,0.000000,0.0,0.018570,0.000000,0.000000,0.000000,0.026918,0.012496,0.052892,0.040105,0.000000,0.000000,0.000000,0.000000
29996,29997.0,1.142857,0.000,0.0,0.000000,1.0,1.000000,1.100000,1.000000,1.142857,0.0,0.000000,0.0,0.029595,0.000000,0.000000,0.000000,0.015176,0.018914,0.023787,0.009680,0.034745,0.000000,0.000000,0.000000
29997,29998.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.013855,0.000000,0.000000,0.000000,0.010758,0.040342,0.022005,0.019560,0.015892,0.000000,0.000000,0.000000
29998,29999.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.0,0.092251,0.000000,0.000000,0.000000,0.092251,0.011070,0.092251,0.073801,0.000000,0.000000,0.000000,0.000000


In [ ]:
data.filter(like="c_ctd").columns

Index(['c_ctdTrx_month_1', 'c_ctdTrx_month_10', 'c_ctdTrx_month_11',
       'c_ctdTrx_month_12', 'c_ctdTrx_month_2', 'c_ctdTrx_month_3',
       'c_ctdTrx_month_4', 'c_ctdTrx_month_5', 'c_ctdTrx_month_6',
       'c_ctdTrx_month_7', 'c_ctdTrx_month_8', 'c_ctdTrx_month_9'],
      dtype='object')

In [ ]:
data

,id_codCliente,c_ctdTrx_month_1,c_ctdTrx_month_10,c_ctdTrx_month_11,c_ctdTrx_month_12,c_ctdTrx_month_2,c_ctdTrx_month_3,c_ctdTrx_month_4,c_ctdTrx_month_5,c_ctdTrx_month_6,c_ctdTrx_month_7,c_ctdTrx_month_8,c_ctdTrx_month_9,c_ratingMonto_month_1,c_ratingMonto_month_10,c_ratingMonto_month_11,c_ratingMonto_month_12,c_ratingMonto_month_2,c_ratingMonto_month_3,c_ratingMonto_month_4,c_ratingMonto_month_5,c_ratingMonto_month_6,c_ratingMonto_month_7,c_ratingMonto_month_8,c_ratingMonto_month_9
0,1.0,1.000000,1.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,0.036963,0.012931,0.000000,0.000000,0.007129,0.015218,0.038929,0.006212,0.030324,0.009807,0.022884,0.017712
1,2.0,1.066667,1.125,1.0,1.166667,1.0,1.000000,1.090909,1.000000,1.000000,1.0,1.000000,1.0,0.008733,0.001440,0.002662,0.002839,0.002498,0.002814,0.003436,0.002868,0.004256,0.003015,0.004821,0.011069
2,3.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.0,0.041763,0.000000,0.000000,0.000000,0.107143,0.042051,0.011377,0.008353,0.018433,0.052707,0.070853,0.000000
3,4.0,1.000000,1.000,1.0,1.066667,1.0,1.095238,1.166667,1.000000,1.000000,1.0,1.300000,1.0,0.004876,0.002249,0.004774,0.003082,0.003483,0.006016,0.002794,0.002507,0.002151,0.002168,0.017737,0.003091
4,5.0,1.000000,1.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.090909,1.0,0.012017,0.016805,0.000000,0.000000,0.085495,0.014337,0.022142,0.004432,0.000919,0.001937,0.011503,0.008077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.285714,0.000000,0.0,0.000000,0.0,0.018570,0.000000,0.000000,0.000000,0.026918,0.012496,0.052892,0.040105,0.000000,0.000000,0.000000,0.000000
29996,29997.0,1.142857,0.000,0.0,0.000000,1.0,1.000000,1.100000,1.000000,1.142857,0.0,0.000000,0.0,0.029595,0.000000,0.000000,0.000000,0.015176,0.018914,0.023787,0.009680,0.034745,0.000000,0.000000,0.000000
29997,29998.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.013855,0.000000,0.000000,0.000000,0.010758,0.040342,0.022005,0.019560,0.015892,0.000000,0.000000,0.000000
29998,29999.0,1.000000,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.0,0.092251,0.000000,0.000000,0.000000,0.092251,0.011070,0.092251,0.073801,0.000000,0.000000,0.000000,0.000000


In [ ]:
#Restringimos los registros a solo aquellos clientes que en  10 meses 
#han tenido movimientos en al menos cinco** meses
#Y evitar a aquellos que son nuevos o los que han dejado de tener movimiento tiempo atras
df_all=data[data[['c_ctdTrx_month_10', 'c_ctdTrx_month_11',
       'c_ctdTrx_month_12', 'c_ctdTrx_month_3',
       'c_ctdTrx_month_4', 'c_ctdTrx_month_5', 'c_ctdTrx_month_6',
       'c_ctdTrx_month_7', 'c_ctdTrx_month_8', 'c_ctdTrx_month_9',"c_ctdTrx_month_1",
                 "c_ctdTrx_month_2"]].apply(lambda x:sum([y!=0 for y in x])>5,axis=1)]

In [ ]:
data.shape

(30000, 25)

In [ ]:

df_all.shape[0]/data.shape[0]

0.9069666666666667

In [ ]:
olap

,id_codCliente,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4,target
0,24041,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN,1
1,9040,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1,1
2,18233,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1,0
3,7226,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2,0
4,21617,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,253,Rango6,Rango2,0,1,271.0,Rango1,Rango3,Rango1,0,Rango1,Rango2,Rango1,Rango1,1
29996,13378,Rango6,Rango1,0,1,271.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango4,0
29997,12503,Rango3,Rango1,1,1,255.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,1
29998,22468,Rango5,Rango3,1,1,255.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1,0


In [ ]:
#Eliminamos el movimiento mas reciente registrado 
#Para poder predecir si el siguiente mes dejará el servicio
df_all=df_all.drop(columns=["c_ctdTrx_month_1","c_ratingMonto_month_1"])

#Para poder predecir si en dos meses dejará el servicio
# Falta por modificar df_all=df_all.drop(columns=["c_ctdTrx_month_1","c_ratingMonto_month_1"])

In [ ]:
df_all=df_all.merge(olap,on="id_codCliente",how="left")

In [ ]:
completitud(df_all)

,columna,total,completitud
0,v_saldoTcEntidad1_x,5669,79.164982
1,v_saldoTcEntidad4,5669,79.164982
2,v_saldoTcEntidad4_x,5669,79.164982
3,v_saldoTcEntidad2_x,5669,79.164982
4,v_saldoTcEntidad2_y,5669,79.164982
...,...,...,...
60,v_flagLimaProvCliente_x,0,100.000000
61,v_rangoCtdProdAct_x,0,100.000000
62,v_rangoCtdProdPas_x,0,100.000000
63,c_ratingMonto_month_12,0,100.000000


In [ ]:
df_all.describe()

,id_codCliente,c_ctdTrx_month_10,c_ctdTrx_month_11,c_ctdTrx_month_12,c_ctdTrx_month_2,c_ctdTrx_month_3,c_ctdTrx_month_4,c_ctdTrx_month_5,c_ctdTrx_month_6,c_ctdTrx_month_7,c_ctdTrx_month_8,c_ctdTrx_month_9,c_ratingMonto_month_10,c_ratingMonto_month_11,c_ratingMonto_month_12,c_ratingMonto_month_2,c_ratingMonto_month_3,c_ratingMonto_month_4,c_ratingMonto_month_5,c_ratingMonto_month_6,c_ratingMonto_month_7,c_ratingMonto_month_8,c_ratingMonto_month_9,v_flagGenero_x,v_flagLimaProvCliente_x,v_ubigeoCliente_x,v_flagBxi_x,target_x,v_flagGenero_y,v_flagLimaProvCliente_y,v_ubigeoCliente_y,v_flagBxi_y,target_y,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_flagBxi,target
count,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27209.000000,27205.000000,27209.000000,27209.000000,27209.000000,27209.000000,27205.000000,27209.000000,27209.000000,27209.000000,27209.000000,27205.000000,27209.000000,27209.000000
mean,14840.954574,0.601596,0.479002,0.321869,1.067262,1.065906,1.065436,1.066817,1.066571,0.950516,0.836523,0.720472,0.008645,0.005851,0.002763,0.027895,0.027660,0.026654,0.027244,0.027395,0.021583,0.016056,0.011847,0.547723,0.792054,237.562838,0.826601,0.701643,0.547723,0.792054,237.562838,0.826601,0.701643,0.547723,0.792054,237.562838,0.826601,0.701643
std,8632.841778,0.545615,0.558735,0.501472,0.251784,0.221683,0.223083,0.245417,0.233487,0.397179,0.486577,0.530998,0.019409,0.014867,0.008568,0.044626,0.042656,0.043788,0.044749,0.044746,0.039743,0.030697,0.024637,0.497726,0.405845,77.115362,0.378598,0.457545,0.497726,0.405845,77.115362,0.378598,0.457545,0.497726,0.405845,77.115362,0.378598,0.457545
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000025,0.000032,0.000057,0.000020,0.000140,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,7358.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.006038,0.006459,0.005886,0.005911,0.005801,0.003830,0.001756,0.000000,0.000000,1.000000,243.000000,1.000000,0.000000,0.000000,1.000000,243.000000,1.000000,0.000000,0.000000,1.000000,243.000000,1.000000,0.000000
50%,14783.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.002621,0.000000,0.000000,0.013549,0.014207,0.012939,0.013094,0.013153,0.009803,0.007028,0.004686,1.000000,1.000000,258.000000,1.000000,1.000000,1.000000,1.000000,258.000000,1.000000,1.000000,1.000000,1.000000,258.000000,1.000000,1.000000
75%,22285.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.009896,0.006871,0.002433,0.031345,0.031849,0.029751,0.030069,0.030324,0.023424,0.017848,0.013375,1.000000,1.000000,275.000000,1.000000,1.000000,1.000000,1.000000,275.000000,1.000000,1.000000,1.000000,1.000000,275.000000,1.000000,1.000000
max,30000.000000,7.000000,14.000000,7.272727,13.000000,7.000000,7.000000,13.000000,9.500000,10.500000,10.000000,9.000000,0.756847,0.507624,0.416815,0.951159,0.887701,0.939926,0.859211,0.840336,0.819187,0.789238,0.800681,1.000000,1.000000,418.000000,1.000000,1.000000,1.000000,1.000000,418.000000,1.000000,1.000000,1.000000,1.000000,418.000000,1.000000,1.000000


In [ ]:
df_all

,id_codCliente,c_ctdTrx_month_10,c_ctdTrx_month_11,c_ctdTrx_month_12,c_ctdTrx_month_2,c_ctdTrx_month_3,c_ctdTrx_month_4,c_ctdTrx_month_5,c_ctdTrx_month_6,c_ctdTrx_month_7,c_ctdTrx_month_8,c_ctdTrx_month_9,c_ratingMonto_month_10,c_ratingMonto_month_11,c_ratingMonto_month_12,c_ratingMonto_month_2,c_ratingMonto_month_3,c_ratingMonto_month_4,c_ratingMonto_month_5,c_ratingMonto_month_6,c_ratingMonto_month_7,c_ratingMonto_month_8,c_ratingMonto_month_9,v_rangoEdad_x,v_rangoIngreso_x,v_flagGenero_x,v_flagLimaProvCliente_x,v_ubigeoCliente_x,v_rangoCtdProdAct_x,v_rangoCtdProdPas_x,v_rangoCtdProdSeg_x,v_flagBxi_x,v_saldoTcEntidad1_x,v_saldoTcEntidad2_x,v_saldoTcEntidad3_x,v_saldoTcEntidad4_x,target_x,v_rangoEdad_y,v_rangoIngreso_y,v_flagGenero_y,v_flagLimaProvCliente_y,v_ubigeoCliente_y,v_rangoCtdProdAct_y,v_rangoCtdProdPas_y,v_rangoCtdProdSeg_y,v_flagBxi_y,v_saldoTcEntidad1_y,v_saldoTcEntidad2_y,v_saldoTcEntidad3_y,v_saldoTcEntidad4_y,target_y,v_rangoEdad,v_rangoIngreso,v_flagGenero,v_flagLimaProvCliente,v_ubigeoCliente,v_rangoCtdProdAct,v_rangoCtdProdPas,v_rangoCtdProdSeg,v_flagBxi,v_saldoTcEntidad1,v_saldoTcEntidad2,v_saldoTcEntidad3,v_saldoTcEntidad4,target
0,1.0,1.000,0.0,0.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,0.012931,0.000000,0.000000,0.007129,0.015218,0.038929,0.006212,0.030324,0.009807,0.022884,0.017712,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,1,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,1,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,1
1,2.0,1.125,1.0,1.166667,1.0,1.000000,1.090909,1.0,1.000000,1.0,1.000000,1.0,0.001440,0.002662,0.002839,0.002498,0.002814,0.003436,0.002868,0.004256,0.003015,0.004821,0.011069,Rango6,Rango2,1,1,241.0,Rango1,Rango3,Rango1,0,Rango6,Rango1,Rango1,Rango1,1,Rango6,Rango2,1,1,241.0,Rango1,Rango3,Rango1,0,Rango6,Rango1,Rango1,Rango1,1,Rango6,Rango2,1,1,241.0,Rango1,Rango3,Rango1,0,Rango6,Rango1,Rango1,Rango1,1
2,3.0,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.0,0.000000,0.000000,0.000000,0.107143,0.042051,0.011377,0.008353,0.018433,0.052707,0.070853,0.000000,Rango6,Rango5,0,1,253.0,Rango1,Rango2,Rango2,0,Rango6,Rango2,Rango1,Rango2,0,Rango6,Rango5,0,1,253.0,Rango1,Rango2,Rango2,0,Rango6,Rango2,Rango1,Rango2,0,Rango6,Rango5,0,1,253.0,Rango1,Rango2,Rango2,0,Rango6,Rango2,Rango1,Rango2,0
3,4.0,1.000,1.0,1.066667,1.0,1.095238,1.166667,1.0,1.000000,1.0,1.300000,1.0,0.002249,0.004774,0.003082,0.003483,0.006016,0.002794,0.002507,0.002151,0.002168,0.017737,0.003091,Rango6,Rango1,1,0,212.0,Rango1,Rango1,Rango2,0,Rango1,Rango1,Rango1,Rango1,1,Rango6,Rango1,1,0,212.0,Rango1,Rango1,Rango2,0,Rango1,Rango1,Rango1,Rango1,1,Rango6,Rango1,1,0,212.0,Rango1,Rango1,Rango2,0,Rango1,Rango1,Rango1,Rango1,1
4,5.0,1.000,0.0,0.000000,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.090909,1.0,0.016805,0.000000,0.000000,0.085495,0.014337,0.022142,0.004432,0.000919,0.001937,0.011503,0.008077,Rango6,Rango1,1,1,271.0,Rango1,Rango1,Rango3,1,Rango2,Rango1,Rango5,Rango1,0,Rango6,Rango1,1,1,271.0,Rango1,Rango1,Rango3,1,Rango2,Rango1,Rango5,Rango1,0,Rango6,Rango1,1,1,271.0,Rango1,Rango1,Rango3,1,Rango2,Rango1,Rango5,Rango1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27204,29993.0,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.0,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.018325,0.027909,0.029226,0.027646,0.266983,0.000000,0.000000,0.000000,Rango1,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,0,Rango1,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,0,Rango1,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,0
27205,29995.0,0.000,0.0,0.000000,1.0,1.000000,1.000000,1.0,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.036